<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
import os
import sys
import pickle
import openslide
import matplotlib
import numpy as np
from PIL import Image
import tensorflow as tf
rootdir_src = "/space/ponzio/teaching-MLinAPP/src/"
dir_slides = "/space/ponzio/CRC_WSIs/"
dir_tiles = "/space/ponzio/CRC_WSIs/CRC_WSIs_tiles"
sys.path.append(rootdir_src)
from annotation_utils import get_annotation_mask
from wsi_utils import filt_tile_placeholders
from wsi_utils import get_heatmap

In [ ]:
img_id = 13
img_fname = os.path.join(dir_slides, "{}.svs".format(img_id))
xml_fname = os.path.join(dir_slides, "{}.xml".format(img_id))
tiles_fname = os.path.join(dir_tiles, "{}_tile_placeholders.pickle".format(img_id))
slide = openslide.OpenSlide(img_fname)
mask = get_annotation_mask(xml_fname, slide, 1/50, "red")

In [ ]:
Image.fromarray(mask)

In [ ]:
with open(tiles_fname, "rb") as fp:
    tiles = pickle.load(fp)
heatmap_NH, segmentation = get_heatmap(tiles,
                                       class_to_map=1,
                                       level_downsample=1/50,
                                       num_classes=2,
                                       threshold=0.5,
                                       colormap=matplotlib.cm.get_cmap('Reds'),
                                       tile_placeholders_mapping_key='prediction')

In [ ]:
heatmap_NH

In [ ]:
Image.fromarray(segmentation)

In [ ]:
segmentation[segmentation!=0] = 255
Image.fromarray(segmentation)

In [ ]:
ground_thruth = np.ravel(mask)/255
preds = np.ravel(segmentation)/255

In [ ]:
m = tf.keras.metrics.MeanIoU(num_classes=2)
m.update_state(ground_thruth, preds)
m.result().numpy()